In [ ]:
!apt update > /dev/null
!apt install chromium-chromedriver -y > /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.binary_location = "/usr/bin/chromium-browser"
driver = webdriver.Chrome(options=options)

base_url = "https://mc.gov.sa/ar/Regulations/Pages/"
start_url = base_url + "default.aspx"

all_links = set()
visited_pages = set()

try:
    driver.get(start_url)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "orderDropDownList"))
    )
    select = Select(driver.find_element(By.ID, "orderDropDownList"))
    select.select_by_value("1")
    time.sleep(2)

    while True:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#regulationsContainer a.rule-clickable-part"))
        )
        links = driver.find_elements(By.CSS_SELECTOR, "#regulationsContainer a.rule-clickable-part")
        for link in links:
            href = link.get_attribute("href")
            if href:
                all_links.add(href)

        pager_links = driver.find_elements(By.CSS_SELECTOR, "#pagerContainer ul.pagination li a[pagenum]")
        next_page = None
        for a in pager_links:
            pagenum = a.get_attribute("pagenum")
            if pagenum and pagenum not in visited_pages:
                next_page = a
                visited_pages.add(pagenum)
                break

        if next_page:
            driver.execute_script("arguments[0].scrollIntoView();", next_page)
            driver.execute_script("window.scrollBy(0, 200);")
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", next_page)
            WebDriverWait(driver, 10).until(EC.staleness_of(links[0]))
        else:
            break

    print(f"✅ Total links collected: {len(all_links)}")

except Exception as e:
    print(f"❌ Error: {e}")
finally:
    driver.quit()

# 👇 This is important to make the variable available in the next cell
all_links = sorted(all_links)


✅ Total links collected: 67


In [54]:
import os
import re
import time
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def apply_rtl(paragraph):
    pPr = paragraph._element.get_or_add_pPr()
    for tag in ['w:bidi', 'w:textDirection']:
        for child in pPr.findall(qn(tag)):
            pPr.remove(child)
    bidi = OxmlElement('w:bidi')
    bidi.set(qn('w:val'), '1')
    text_direction = OxmlElement('w:textDirection')
    text_direction.set(qn('w:val'), 'rtl')
    pPr.insert(0, text_direction)
    pPr.insert(0, bidi)

def clean_meta_info(text):
    return re.sub(r'\|\s*\|', '|', text).replace(' | ', '|').strip()

def extract_section(driver, section_name, law_title):
    try:
        header = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, f"//h3[contains(., '{section_name}')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", header)
        time.sleep(0.5)

        try:
            driver.execute_script("arguments[0].click();", header)
            time.sleep(2)
        except:
            pass

        html = ""
        try:
            container = header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'regulation-container')]")
            html = container.get_attribute("outerHTML")
        except:
            rules_title_div = header.find_element(By.XPATH, "./ancestor::div[contains(@class, 'rules-title-container')]")
            html = rules_title_div.get_attribute("outerHTML")
            try:
                sibling = rules_title_div.find_element(By.XPATH, "./following-sibling::*[1]")
                html += sibling.get_attribute("outerHTML")
            except:
                pass

        soup = BeautifulSoup(html, "html.parser")
        meta_div = soup.find("div", class_="rule-info-container")
        meta_info = clean_meta_info(meta_div.get_text(separator=" | ", strip=True)) if meta_div else ""

        content = []
        articles = soup.find_all("div", class_="rules-article-container")
        if not articles:
            return None

        for art in articles:
            parent_h3 = art.find_previous("h3", class_="ui-accordion-header")
            if parent_h3:
                chapter_title = parent_h3.get_text(strip=True)
                if not any(c[1] == chapter_title for c in content if c[0] == "chapter"):
                    content.append(("chapter", chapter_title))

            title = art.find("h4")
            if title:
                content.append(("article_title", title.get_text(strip=True)))

            body = []
            for child in art.children:
                if child.name and child.name != 'h4':
                    if child.name in ['ul', 'ol']:
                        for li in child.find_all('li', recursive=False):
                            body.append(f"• {li.get_text(strip=True)}")
                    else:
                        text = child.get_text(separator="\n", strip=True)
                        if text:
                            body.append(text)
            if body:
                content.append(("article_body", "\n".join(body)))
            content.append(("empty", ""))

        return meta_info, content

    except Exception as e:
        print(f"⚠️ Could not process section '{section_name}': {e}")
        return None

def save_docx(law_title, section_name, meta_info, content):
    doc = Document()
    style = doc.styles['Normal']
    style.font.name = 'Arial'
    style.font.size = Pt(12)

    p = doc.add_paragraph(f"{law_title} - {section_name}")
    p.style = doc.styles['Heading 1']
    apply_rtl(p)

    if meta_info:
        meta = doc.add_paragraph(meta_info)
        apply_rtl(meta)
        doc.add_paragraph()

    for kind, text in content:
        if kind == "chapter":
            p = doc.add_paragraph(text)
            p.style = doc.styles['Heading 2']
            apply_rtl(p)
        elif kind == "article_title":
            p = doc.add_paragraph(text)
            p.style = doc.styles['Heading 3']
            apply_rtl(p)
        elif kind == "article_body":
            for line in text.split("\n"):
                p = doc.add_paragraph(line)
                apply_rtl(p)
        elif kind == "empty":
            doc.add_paragraph()

    filename = f"{law_title} - {section_name}.docx"
    path = os.path.join("exported_docs", filename)
    doc.save(path)
    print(f"💾 Saved: {path}")

# --- MAIN ---
chromedriver_autoinstaller.install()
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.binary_location = "/usr/bin/chromium-browser"
driver = webdriver.Chrome(options=options)

os.makedirs("exported_docs", exist_ok=True)

for url in all_links:
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            lambda d: d.find_element(By.CSS_SELECTOR, "h2.pagetitle").text.strip() != ""
        )
        law_title = driver.find_element(By.CSS_SELECTOR, "h2.pagetitle").text.strip()
        print(f"\n📄 Law title: {law_title}")

        for section in ["النظام", "اللائحة"]:
            print(f"📖 Processing section: {section}")
            result = extract_section(driver, section, law_title)
            if result:
                meta_info, content = result
                save_docx(law_title, section, meta_info, content)
            else:
                print(f"⚠️ Skipped section: {section}")
    except Exception as e:
        print(f"❌ Error processing {url}: {e}")

driver.quit()
print("✨ Done.")



📄 Law title: نظام الشركات
📖 Processing section: النظام
💾 Saved: exported_docs/نظام الشركات - النظام.docx
📖 Processing section: اللائحة
⚠️ Could not process section 'اللائحة': Message: 
Stacktrace:
#0 0x5766097e545a <unknown>
#1 0x57660928a760 <unknown>
#2 0x5766092dc127 <unknown>
#3 0x5766092dc321 <unknown>
#4 0x57660932ae94 <unknown>
#5 0x576609301e7d <unknown>
#6 0x57660932830b <unknown>
#7 0x576609301c23 <unknown>
#8 0x5766092ce4a5 <unknown>
#9 0x5766092cf111 <unknown>
#10 0x5766097a9f1b <unknown>
#11 0x5766097ade19 <unknown>
#12 0x576609790ac9 <unknown>
#13 0x5766097ae9c8 <unknown>
#14 0x57660977534f <unknown>
#15 0x5766097d2a28 <unknown>
#16 0x5766097d2c06 <unknown>
#17 0x5766097e4336 <unknown>
#18 0x7ba3506e1ac3 <unknown>

⚠️ Skipped section: اللائحة

📄 Law title: اللائحة التنفيذية لنظام التحكيم التجاري
📖 Processing section: النظام
⚠️ Could not process section 'النظام': Message: 
Stacktrace:
#0 0x5766097e545a <unknown>
#1 0x57660928a760 <unknown>
#2 0x5766092dc127 <unknown>
#3 0